# DSA 6000
# TEAM 10
# Los Angeles crime data

# Data Cleaning

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 50
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')
import seaborn as sns
import sklearn
import pickle

## Loading Crime Dataset

In [ ]:
df = pd.read_csv('/content/crime_data.csv')

In [ ]:
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,0444 0913,36,F,B,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,0416 1822 1414,25,M,H,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,1501,0,X,X,726.0,POLICE FACILITY,NaN,NaN,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,76,F,W,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,31,X,X,409.0,BEAUTY SUPPLY STORE,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [ ]:
df.shape

(829778, 28)

> The uncleaned data has over 800 thousand instances and 28 columns

## Converting `date occured` to Time Series Index

In [ ]:
# setting 'date occured' column as datetime object
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])

###Sorting df linearly

## Checking empty values for further cleaning

In [ ]:
# checking for null values
df.isna().sum()

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           114856
Vict Age               0
Vict Sex          109299
Vict Descent      109307
Premis Cd             10
Premis Desc          492
Weapon Used Cd    540459
Weapon Desc       540459
Status                 0
Status Desc            0
Crm Cd 1              10
Crm Cd 2          768750
Crm Cd 3          827720
Crm Cd 4          829717
LOCATION               0
Cross Street      697270
LAT                    0
LON                    0
dtype: int64

In [ ]:
# dropping columns with a majority of missing values and unnessary informatoion
df = df.drop(axis=1, columns=['Mocodes',
                              'Weapon Used Cd',
                              'Weapon Desc',
                              'Crm Cd 1', # same as feature 'Crime Code'
                              'Crm Cd 2',
                              'Crm Cd 3',
                              'Crm Cd 4',
                              'Cross Street',
                             'Date Rptd',])

## Data Cleaning

### Since premise code and description is only missing few values replacing with appropriate values

In [ ]:
df['Premis Cd'].fillna(0, inplace = True)
df['Premis Desc'].fillna('UNKNOWN', inplace = True)

In [ ]:
df.isna().sum()

DR_NO                0
DATE OCC             0
TIME OCC             0
AREA                 0
AREA NAME            0
Rpt Dist No          0
Part 1-2             0
Crm Cd               0
Crm Cd Desc          0
Vict Age             0
Vict Sex        109299
Vict Descent    109307
Premis Cd            0
Premis Desc          0
Status               0
Status Desc          0
LOCATION             0
LAT                  0
LON                  0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829778 entries, 0 to 829777
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   DR_NO         829778 non-null  int64         
 1   DATE OCC      829778 non-null  datetime64[ns]
 2   TIME OCC      829778 non-null  int64         
 3   AREA          829778 non-null  int64         
 4   AREA NAME     829778 non-null  object        
 5   Rpt Dist No   829778 non-null  int64         
 6   Part 1-2      829778 non-null  int64         
 7   Crm Cd        829778 non-null  int64         
 8   Crm Cd Desc   829778 non-null  object        
 9   Vict Age      829778 non-null  int64         
 10  Vict Sex      720479 non-null  object        
 11  Vict Descent  720471 non-null  object        
 12  Premis Cd     829778 non-null  float64       
 13  Premis Desc   829778 non-null  object        
 14  Status        829778 non-null  object        
 15  Status Desc   829

In [ ]:
df.describe().apply(lambda s: s.apply('{0:.2f}'.format))

,DR_NO,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,LAT,LON
count,829778.00,829778.00,829778.00,829778.00,829778.00,829778.00,829778.00,829778.00,829778.00,829778.00
mean,216295184.53,1335.88,10.71,1117.38,1.41,500.83,29.79,305.82,33.98,-118.03
std,10883328.76,653.95,6.09,609.36,0.49,207.79,21.78,216.77,1.78,6.17
min,817.00,1.00,1.00,101.00,1.00,110.00,-3.00,0.00,0.00,-118.67
25%,210208424.25,900.00,6.00,621.00,1.00,331.00,6.00,101.00,34.01,-118.43
50%,220123790.00,1415.00,11.00,1142.00,1.00,442.00,31.00,203.00,34.06,-118.32
75%,222014958.50,1900.00,16.00,1615.00,2.00,626.00,45.00,501.00,34.16,-118.27
max,239916487.00,2359.00,21.00,2199.00,2.00,956.00,120.00,976.00,34.33,0.00


## Victim sex data cleaning

In [ ]:
df['Vict Sex'].value_counts()

M    342350
F    305479
X     72559
H        90
-         1
Name: Vict Sex, dtype: int64

In [ ]:
# replacing H, X and missing values with X indicating unknown
df['Vict Sex'].replace(to_replace=['H', '-', 'N'], value='X', inplace = True)
df['Vict Sex'].fillna('X', inplace= True)

In [ ]:
df['Vict Sex'].unique()

array(['F', 'M', 'X'], dtype=object)

## Victim Age cleaning

In [ ]:
df[df['Vict Age'] < 0]['Vict Age'].count()

71

> A person cannot be under the age of 0 so replacing all instances less than 0 with 0

In [ ]:
df['Vict Age'].unique()

array([ 36,  25,   0,  76,  31,  23,  29,  35,  41,  24,  34,  46,  66,
        40,  27,  62,  43,  71,  50,  19,  51,  33,  69,  39,  57,  78,
        52,  38,  55,  44,  18,  54,  22,  28,  42,  56,  67,  37,  60,
        61,  59,  32,  30,  45,  20,  15,  58,  47,  48,  26,  21,  64,
        75,  12,  49,  68,  14,  13,  10,  53,  74,  65,  63,   8,  16,
        72,  70,   9,  90,  85,  81,  79,  94,  73,  11,  17,  80,   5,
        82,   2,  77,  84,  88,  96,  99,   7,  86,  92,   3,  83,  87,
         6,  -1,  89,   4,  93,  98,  91,  95,  97, 120,  -2,  -3])

> It appears that there are bunch of negative ages which is impossible

### Replacing negative ages with 0

In [ ]:
df['Vict Age'] = np.where((df['Vict Age'] < 0), 0,df['Vict Age'])

In [ ]:
df[df['Vict Age'] < 0]['Vict Age'].count()

0

In [ ]:
df['Vict Age'].isna().sum()

0

## Victim Desent

In [ ]:
df['Vict Descent'].value_counts()

H    254505
W    168980
B    118097
X     80124
O     65732
A     18161
K      4422
F      3450
C      3192
J      1151
V       858
I       780
Z       412
P       222
U       167
D        62
G        58
L        50
S        46
-         2
Name: Vict Descent, dtype: int64

In [ ]:
descent_dict ={'A': 'Other Asian', 'B': 'Black', 'C': 'Chinese',
               'D': 'Cambodian', 'F': 'Filipino', 'G': 'Guamanian',
               'H': 'Latinx', 'I': 'Native', 'J': 'Japanese',
               'K': 'Korean', 'L': 'Laotian', 'O': 'Other',
               'P': 'Pacific Islander', 'S': 'Samoan', 'U': 'Hawaiian',
               'V': 'Vietnamese', 'W': 'White',  'X': 'Unknown', 'Z':'Asian Indian'}
df['Vict Descent'].fillna('X', inplace= True)

In [ ]:
df['Vict Descent'].replace(to_replace=['-'], value='X', inplace = True)

## Fully Cleaned Data Set

In [ ]:
df.isna().sum()

DR_NO           0
DATE OCC        0
TIME OCC        0
AREA            0
AREA NAME       0
Rpt Dist No     0
Part 1-2        0
Crm Cd          0
Crm Cd Desc     0
Vict Age        0
Vict Sex        0
Vict Descent    0
Premis Cd       0
Premis Desc     0
Status          0
Status Desc     0
LOCATION        0
LAT             0
LON             0
dtype: int64

In [ ]:
df['Crm Cd Desc'].value_counts().count()

138

In [ ]:
def dictionary_maker(key_feature, value_feature):
    '''
    this function takes in two features
    and returns a dictionary with the unique values
    from the two features as key and values
    '''
    # dictionary key
    key= list(df[key_feature].value_counts().index)

    #dictionary value
    value= list(df[value_feature].value_counts().index)

    #dictionary
    dictionary = {key[i]:value[i] for i in range(len(key))}

    return dictionary

In [ ]:
area_dict = dictionary_maker('AREA', 'AREA NAME')
crime_dict = dictionary_maker('Crm Cd', 'Crm Cd Desc')

In [ ]:
area_dict

{1: 'Central',
 12: '77th Street',
 14: 'Pacific',
 3: 'Southwest',
 6: 'Hollywood',
 18: 'Southeast',
 20: 'Olympic',
 13: 'Newton',
 15: 'N Hollywood',
 7: 'Wilshire',
 2: 'Rampart',
 8: 'West LA',
 11: 'Northeast',
 9: 'Van Nuys',
 10: 'West Valley',
 5: 'Harbor',
 21: 'Topanga',
 17: 'Devonshire',
 19: 'Mission',
 4: 'Hollenbeck',
 16: 'Foothill'}

In [ ]:
crime_dict

{510: 'VEHICLE - STOLEN',
 624: 'BATTERY - SIMPLE ASSAULT',
 354: 'THEFT OF IDENTITY',
 330: 'BURGLARY FROM VEHICLE',
 740: 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)',
 310: 'BURGLARY',
 230: 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 440: 'THEFT PLAIN - PETTY ($950 & UNDER)',
 626: 'INTIMATE PARTNER - SIMPLE ASSAULT',
 420: 'THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)',
 331: 'THEFT FROM MOTOR VEHICLE - GRAND ($950.01 AND OVER)',
 210: 'ROBBERY',
 341: 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD',
 745: 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)',
 442: 'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
 930: 'CRIMINAL THREATS - NO WEAPON DISPLAYED',
 761: 'BRANDISH WEAPON',
 888: 'TRESPASSING',
 236: 'INTIMATE PARTNER - AGGRAVATED ASSAULT',
 901: 'VIOLATION OF RESTRAINING ORDER',
 480: 'BIKE - STOLEN',
 956: 'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD',
 946: 'OTHER MISCELLANEOUS CRIME',
 900: 'VIOLATION OF COURT ORDER',
 662: 'BUNCO, GRAND THEFT',
 220:

In [ ]:
df['Status'].value_counts()

IC    663996
AO     89356
AA     72296
JA      2693
JO      1433
CC         4
Name: Status, dtype: int64

In [ ]:
df['Status Desc'].replace(to_replace=['CC'], value='UNK', inplace = True)

In [ ]:
df['Status Desc'].value_counts()

Invest Cont     663996
Adult Other      89356
Adult Arrest     72296
Juv Arrest        2693
Juv Other         1433
UNK                  4
Name: Status Desc, dtype: int64

In [ ]:
df['Premis Desc'].value_counts()
# df[df['Premise Description']=='UNKNOWN']

STREET                                          209976
SINGLE FAMILY DWELLING                          140565
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)    101869
PARKING LOT                                      58096
OTHER BUSINESS                                   39130
                                                 ...  
DEPT OF DEFENSE FACILITY                             2
SEWAGE FACILITY/PIPE                                 2
HORSE RACING/SANTA ANITA PARK*                       2
MTA - SILVER LINE - DOWNTOWN STREET STOPS            2
TRAM/STREETCAR(BOXLIKE WAG ON RAILS)*                1
Name: Premis Desc, Length: 307, dtype: int64

In [ ]:
df['Premis Cd'] = np.where(df['Premis Desc']== 'UNKNOWN', 0.0, df['Premis Cd'])

In [ ]:
df.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Cd', 'Premis Desc', 'Status', 'Status Desc',
       'LOCATION', 'LAT', 'LON'],
      dtype='object')

In [ ]:
df.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Cd', 'Premis Desc', 'Status', 'Status Desc',
       'LOCATION', 'LAT', 'LON'],
      dtype='object')

####  Location Column Cleaning

In [ ]:
# crime_dict
df.head()

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Status,Status Desc,LOCATION,LAT,LON
0,10304468,2020-01-08,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,36,F,B,501.0,SINGLE FAMILY DWELLING,AO,Adult Other,1100 W 39TH PL,34.0141,-118.2978
1,190101086,2020-01-01,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,25,M,H,102.0,SIDEWALK,IC,Invest Cont,700 S HILL ST,34.0459,-118.2545
2,200110444,2020-02-13,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,0,X,X,726.0,POLICE FACILITY,AA,Adult Arrest,200 E 6TH ST,34.0448,-118.2474
3,191501505,2020-01-01,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",IC,Invest Cont,5400 CORTEEN PL,34.1685,-118.4019
4,191921269,2020-01-01,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,X,409.0,BEAUTY SUPPLY STORE,IC,Invest Cont,14400 TITUS ST,34.2198,-118.4468


In [ ]:
df.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Cd', 'Premis Desc', 'Status', 'Status Desc',
       'LOCATION', 'LAT', 'LON'],
      dtype='object')

In [ ]:
# dropping description columns and features not relevant to modeling
cleaned_features=[ 'DATE OCC', 'TIME OCC',
                  'AREA', 'AREA NAME',
                  'Rpt Dist No',
                  'Crm Cd', 'Crm Cd Desc',
                  'Vict Age',
                  'Vict Sex',
                  'Vict Descent',
                  'Premis Cd', 'Premis Desc',
                  'Status', 'Status Desc',
                  'LAT', 'LON']

## Pickling Final Cleaned Dataframe

In [ ]:
cleaned_df = df[cleaned_features]

In [ ]:
cleaned_df.to_pickle("/content/cleaned_df.pkl")